In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import model
import train
import evaluate
import train_util
import data_util.data
import data_util.batcher
import data_util.config
import data_util.preprocess

importlib.reload(train)
importlib.reload(model)
importlib.reload(evaluate)
importlib.reload(train_util)
importlib.reload(data_util.config)
importlib.reload(data_util.data)
importlib.reload(data_util.batcher)
importlib.reload(data_util.preprocess)

from train import *
from evaluate import *
from model import *
from train_util import *
from data_util.data import *
from data_util.batcher import *
from data_util.preprocess import *

In [3]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data_fresh.tsv'
dat = pd.read_csv(data_path, sep='\t')

# fill nas
dat.fillna('', inplace=True)

# train/test split
np.random.seed(111)
dat = dat.sample(frac=1)

train_size = int(.8*dat.shape[0])
train_data = dat[:train_size]
test_data = dat[train_size:]


In [4]:
# Process the the data
def prep_data(df):
    df['Context'] = df['Context'].map(lambda x: article_process_text(x))
    df['TaskSentence'] = df['TaskSentence'].map(lambda x: article_process_text(x))
    df['Summary'] = df['Summary'].map(lambda x: summary_process_text(x))
    return df
train_data = prep_data(train_data)
test_data = prep_data(test_data)

In [5]:
train_data

,#,Task,Context,TaskSentence,Summary,Labeler,NoRequestInContext,Urgent,NotRequest,Unsure/Discuss,RandomNumber
1083,5187,Please give me a shout with,Attached is the file that I use from Storey to...,Please give me a shout with any questions.,contact SENDER with questions,Natalie,1,0,1,,0.053197
384,385,Please respond whether we have your most updat...,"Hi, I'm Shawna with Icon. I am updating our da...",Please respond whether we have your most updat...,check whether information is updated,Rowan,,0,,,0.282344
1348,5452,plz call when you can.,,plz call when you can 415-7827-822 .,call SENDER at 415-7827-822,Natalie,1,0,0,,0.779285
535,1727,"Karen, please call me when you receive this em...",,"Karen, please call me when you receive this em...",call SENDER,Percy,,,1,,0.929963
694,3461,Please look over the attached at your convenie...,I have them revised the document to reflect th...,Please look over the attached at your convenie...,look over attached documents,George,,,,,0.989056
...,...,...,...,...,...,...,...,...,...,...,...
1241,5345,IMAGE To receive our e-mails in a text-only fo...,Your kind of clothes. Online. All the time. IM...,please reply to this message and type change t...,reply to SENDER and type change to text,Natalie,1,0,1,,0.308842
305,306,"In light of this, please make sure your Confir...","Diane or Patrick, I need help from one of you ...","In light of this, please make sure your Confir...",ensure confirmation copy includes the GTC,Rowan,1,0,,,0.580914
247,248,Please review the attached non-standard discou...,"I'm OK on this, with one small change. Please ...",Please review the attached non-standard discou...,review attached letter,Rowan,,0,,,0.149520
181,182,"Please, check the logic of my",Headcount gives the allocations by major busin...,"Please, check the logic of my modifications I ...",check logic of modifications,Rowan,,,,,0.305235


In [6]:
vocab = Vocab.from_vocab_file(os.path.join(config.log_root, 'data/vocab.txt'))

Finished constructing vocabulary of 4654 total words. Last word added: welcome


In [9]:
vocab.word2id('now')

1788

In [7]:
random.seed(123)
T.manual_seed(123)
if T.cuda.is_available():
    T.cuda.manual_seed_all(123)
    
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [63]:
train_mle = "yes"
train_rl = "no"
mle_weight = 1.0
load_model = None
new_lr = None
rl_weight = 1 - mle_weight

opt = Namespace(train_mle = train_mle, 
                train_rl = train_rl, 
                mle_weight = mle_weight, 
                load_model = load_model,
                new_lr = new_lr, 
                rl_weight = rl_weight)

task_batcher = TaskBatcher(
    examples=train_data.to_dict('records')[:20],
    vocab=vocab,
    mode='train',
    batch_size=10,
    single_pass=False
)

val_task_batcher = TaskBatcher( # Batching obj
    examples=train_data.to_dict('records')[:20],
    vocab=vocab, 
    mode='train', 
    batch_size=10, 
    single_pass=False
)


train_processor = TaskTrain(vocab, task_batcher, opt, TaskModel, val_task_batcher)

In [64]:
config.save_model_path = "data/debug"

mle_losses = train_processor.trainIters(n_iters=2, report_every=1, save_every = 1)

iter: 1 mle_loss: 6.134 mle_loss_val: 6.3944
model saved at: 
 data/debug/0000001.tar
iter: 2 mle_loss: 6.666 mle_loss_val: 6.3725
model saved at: 
 data/debug/0000002.tar
iter: 3 mle_loss: 6.108 mle_loss_val: 6.2602
model saved at: 
 data/debug/0000003.tar


In [67]:
orig_model = train_processor.model.state_dict()

In [68]:
orig_model == orig_model

True

Decoding Time

In [69]:

task = "validate"

load_model = os.path.join(config.log_root, "data/debug/0000003.tar") # model directory

opt = Namespace(task = task, load_model = load_model) # opt


# new batcher for evaluation
task_batcher = TaskBatcher( # Batching obj
    examples=test_data.to_dict('records')[:5],
    vocab=vocab, 
    mode='train', 
    batch_size=5, 
    single_pass=True)

eval_processor = TaskEvaluate(vocab, task_batcher, opt, TaskModel) # Evaluation object

example_generator completed reading all examples. No more data.


Exception in thread Thread-37:
Traceback (most recent call last):
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 443, in text_generator
    example = next(example_generator)
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 424, in fill_example_queue
    context, task, summary = next(input_gen) # read the next example from file. article and abstract are both strings.
RuntimeError: generator raised StopIteration



model_path /Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data/debug/0000003.tar


In [70]:
orig_model

OrderedDict([('encoder.lstm.weight_ih_l0',
              tensor([[-0.0022, -0.0165,  0.0154,  ..., -0.0091,  0.0131,  0.0066],
                      [ 0.0129, -0.0126,  0.0082,  ...,  0.0166, -0.0147,  0.0112],
                      [-0.0104, -0.0044, -0.0191,  ..., -0.0132,  0.0011, -0.0098],
                      ...,
                      [-0.0165, -0.0199, -0.0125,  ..., -0.0194,  0.0072, -0.0136],
                      [-0.0023, -0.0095,  0.0003,  ...,  0.0001, -0.0098,  0.0128],
                      [ 0.0139,  0.0122, -0.0129,  ..., -0.0046, -0.0128,  0.0024]])),
             ('encoder.lstm.weight_hh_l0',
              tensor([[-0.0201,  0.0190, -0.0080,  ..., -0.0151, -0.0123, -0.0063],
                      [ 0.0029, -0.0105, -0.0037,  ..., -0.0006, -0.0161, -0.0067],
                      [ 0.0063,  0.0157, -0.0007,  ..., -0.0061,  0.0025,  0.0095],
                      ...,
                      [-0.0015, -0.0159, -0.0136,  ..., -0.0114,  0.0137, -0.0006],
                 

In [71]:
eval_processor.model.state_dict()

OrderedDict([('encoder.lstm.weight_ih_l0',
              tensor([[-0.0022, -0.0165,  0.0154,  ..., -0.0091,  0.0131,  0.0066],
                      [ 0.0129, -0.0126,  0.0082,  ...,  0.0166, -0.0147,  0.0112],
                      [-0.0104, -0.0044, -0.0191,  ..., -0.0132,  0.0011, -0.0098],
                      ...,
                      [-0.0165, -0.0199, -0.0125,  ..., -0.0194,  0.0072, -0.0136],
                      [-0.0023, -0.0095,  0.0003,  ...,  0.0001, -0.0098,  0.0128],
                      [ 0.0139,  0.0122, -0.0129,  ..., -0.0046, -0.0128,  0.0024]])),
             ('encoder.lstm.weight_hh_l0',
              tensor([[-0.0201,  0.0190, -0.0080,  ..., -0.0151, -0.0123, -0.0063],
                      [ 0.0029, -0.0105, -0.0037,  ..., -0.0006, -0.0161, -0.0067],
                      [ 0.0063,  0.0157, -0.0007,  ..., -0.0061,  0.0025,  0.0095],
                      ...,
                      [-0.0015, -0.0159, -0.0136,  ..., -0.0114,  0.0137, -0.0006],
                 

In [39]:
decoded_sents, ref_sents, article_sents = eval_processor.evaluate_batch()

Summarizing Batch...
INFO:tensorflow:Finished reading dataset in single_pass mode.


In [40]:
len(ref_sents)

5

In [41]:
len(decoded_sents)

5

In [42]:
scores = Rouge().get_scores(decoded_sents, ref_sents, avg = True)

In [43]:
scores

{'rouge-1': {'f': 0.10714285557653062, 'p': 0.2, 'r': 0.07333333333333333},
 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
 'rouge-l': {'f': 0.10714285557653062, 'p': 0.2, 'r': 0.07333333333333333}}

In [44]:
pd.set_option('display.max_colwidth', -1)

In [45]:
df = pd.DataFrame({'ref': ref_sents, 'decoded': decoded_sents})
df.head()

,ref,decoded
0,tell Tom to clarify the cp,Pinnacle . tell
1,determine how renewal contract work will be coordinated,please on
2,forward information to beneficial people,. save to
3,look at definition for diesel index,. can
4,review the modified documents,Ellen please attached


In [ ]:
df.to_csv(os.path.join(config.log_root, 'data/test_results_2.csv'), sep = '\t', header=True, index=False)

In [ ]:
'packet' in vocab._word_to_id

In [ ]:
'interview' in vocab._word_to_id

In [ ]:
# some training examples
pd.DataFrame({'ref': ref_sents, 'decoded': decoded_sents})

In [ ]:
test_data.head()

In [ ]:
test_data

In [ ]:
ref_sents

..
..


In [ ]:
from nltk.corpus import words

In [ ]:
len(words.words())

In [ ]:
import nltk

In [ ]:
for w in nltk.corpus.wordnet.words(): print(w)

In [ ]:
len(list(nltk.corpus.wordnet.words()))